In [1]:
from keras.models import *
from keras.layers import *
from keras.optimizers import RMSprop,SGD,Adadelta,Adam
from keras.preprocessing import image
from keras.callbacks import TensorBoard,History,EarlyStopping,CSVLogger
from keras.applications.vgg19 import VGG19,preprocess_input
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import h5py
import os

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
train_path = './train'
validation_path = './validation'
test_path = './test'
assert(os.path.exists(train_path))
assert(os.path.exists(validation_path))
assert(os.path.exists(test_path))


nb_train_samples = 20000
nb_val_samples = 2424
image_size = (224,224)
batch_size = 128
class_num = 10

In [3]:
def image_preprocess(img_path):

    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    
    return img

In [4]:
def test_model(model,test_path,csv='sample_submission.csv'):
    columns = ['img', 'c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    df = pd.DataFrame(columns=columns)
    
    file_list = os.listdir(test_path)
    for i,file in enumerate(file_list):
        test_data = image_preprocess(test_path + '/' + file)
        y_pred = model.predict(test_data,batch_size=1,verbose=0)
        y_pred = np.clip(y_pred,0.001,0.999)
        y_pred = y_pred[0].tolist()
        df.loc[i] = [file] + y_pred
        
    df.to_csv(csv,index=None)

In [5]:
def ensemble_models(models,model_input):
    outputs = [model(model_input) for model in models]
    
    avrg = average(outputs)
    
    model = Model(inputs=model_input, outputs=avrg, name='ensemble_model')
    
    return model

In [8]:
vgg16_model = load_model('vgg16_final_model.h5')
vgg16_model.name = 'vgg16_model'

vgg19_model = load_model('vgg19_final_model.h5')
vgg19_model.name = 'vgg19_model'

models = [vgg16_model,vgg19_model]

model_input = Input(shape=models[0].input_shape[1:])

ensemble_model = ensemble_models(models,model_input)

ensemble_model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
vgg16_model (Model)             (None, 10)           21207882    input_2[0][0]                    
__________________________________________________________________________________________________
vgg19_model (Model)             (None, 10)           26517578    input_2[0][0]                    
__________________________________________________________________________________________________
average_2 (Average)             (None, 10)           0           vgg16_model[1][0]                
                                                                 vgg19_model[1][0]                
Total para

In [10]:
test_model(ensemble_model,test_path=test_path,csv='vgg16_and_vgg19_submission_0430.csv')

In [ ]:
# model1 = load_model('simple_cnn_final_model.h5')
# model2= load_model('simple_cnn_final_model.h5')

# input1 = model1.input
# input2 = model2.input

# # layer1 = model1.get_layer(index=-2)
# # layer2 = model2.get_layer(index=-2)

# # avrg_layer = average([layer1,layer2])
# # out = Dense(10, activation='softmax')(avrg_layer)
# output1 = model1.output
# output2 = model2.output

# out = Average()([output1,output2])

# model = Model(inputs=[input1,input2],outputs=out)
# model1.summary()

In [8]:
def get_features(model):
    image_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)

    train_generator = image_datagen.flow_from_directory(
                                        train_path,
                                        target_size=image_size,
                                        batch_size=batch_size,
                                        class_mode='categorical',
                                        shuffle=False)

    validation_generator = image_datagen.flow_from_directory(
                                        validation_path,
                                        target_size=image_size,
                                        batch_size=batch_size,                                    
                                        class_mode='categorical',
                                        shuffle=False)
    
    train_data = model.predict_generator(train_generator,nb_train_samples)
    train_label = train_generator.classes
    
    validation_data = model.predict_generator(validation_generator,nb_val_samples)
    validation_label = validation_generator.classes
    
    return train_data,train_label,validation_data,validation_label

In [9]:
vgg16_model = load_model('vgg16_final_model.h5')
vgg16_model.name = 'vgg16_model'

vgg19_model = load_model('vgg19_final_model.h5')
vgg19_model.name = 'vgg19_model'

train_data = []
train_label = []
validation_data = []
validation_label = []

for model in [vgg16_model,vgg19_model]:
    td,tl,vd,vl = get_features(model)
    train_data.append(td)
    train_label.append(tl)
    validation_data.append(vd)
    validation_label.append(vl)
    
train_data = np.concatenate(train_data, axis=1)
train_label = np.concatenate(train_label, axis=1)
validation_data = np.concatenate(validation_data, axis=1)
validation_label = np.concatenate(validation_label, axis=1)

(train_data,train_label) = shuffle(train_data,train_label)

model_input = Input(shape=train_data.shape[1:])
x = Dropout(0.5)(model_input)
x = Dense(10, activation='softmax')(x)
merge_model = Model(inputs=model_input,outputs=x)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_data,train_label,batch_size=batch_size,epochs=10,
          validation_data=(validation_data,validation_label),
          shuffle=True)

Found 20000 images belonging to 10 classes.
Found 2424 images belonging to 10 classes.
Found 20000 images belonging to 10 classes.
Found 2424 images belonging to 10 classes.


AxisError: axis 1 is out of bounds for array of dimension 1

In [14]:
print(train_data.shape)
print(validation_data.size())

(2547808, 20)


AttributeError: 'list' object has no attribute 'size'

In [ ]:
# test_model(ensenble_model,test_path=test_path,csv='vgg16_and_vgg19_submission.csv')